# Severstel Steel Defect Detection

In [0]:
!pip3 install --user kaggle --upgrade

     |████████████████████████████████| 61kB 7.2MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=aef43e89029694351c549d3c13b5875bbe84ddc4d00413cff16071a9e285a059
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kiranscaria","key":"3d0516c994f8dc33547e7083a17b1e5e"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [0]:
!kaggle competitions download -c severstal-steel-defect-detection

 72% 5.00M/6.91M [00:00<00:00, 19.7MB/s]
100% 6.91M/6.91M [00:00<00:00, 23.0MB/s]
  0% 0.00/141k [00:00<?, ?B/s]
100% 141k/141k [00:00<00:00, 147MB/s]
100% 1.16G/1.16G [00:16<00:00, 74.1MB/s]
100% 1.16G/1.16G [00:16<00:00, 77.8MB/s]
 92% 119M/129M [00:01<00:00, 66.9MB/s] 
100% 129M/129M [00:01<00:00, 86.2MB/s]


In [0]:
!mkdir data
!mkdir data/train_images
!mkdir data/test-images
!mv -v *.zip data/
!mv -v *.csv data/

renamed 'test_images.zip' -> 'data/test_images.zip'
renamed 'train.csv.zip' -> 'data/train.csv.zip'
renamed 'train_images.zip' -> 'data/train_images.zip'
renamed 'sample_submission.csv' -> 'data/sample_submission.csv'


In [0]:
%%capture
!unzip data/train.csv.zip -d data/
!unzip data/train_images.zip -d data/train_images/
!unzip data/test_images.zip -d data/test_images/

In [0]:
import os
import cv2
import time
import math
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import BatchNorm2d
from torch.nn import functional as F
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.transforms import functional as Ftrans

In [0]:
seed = 42
batch_size = 4
num_epochs = 20
test_percent = 0.20
learning_rate = 3e-4
# Accumulates gradient of four mini-batches
# Effectively making the batch_size equal to accumulation_steps*times the batch_size
accumulation_steps = 4   

num_workers = 4

random.seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_folder = 'data/train_images'

## Download Dataset

## RLE-Mask utility functions

In [0]:
#https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def mask2rle(img):
    '''
    img: numpy array, 1 -> mask, 0 -> background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def make_mask(row_id, df):
    '''Given a row index, return image_id and mask (256, 1600, 4) from the dataframe `df`'''
    fname = df.iloc[row_id].name
    labels = df.iloc[row_id][:4]
    masks = np.zeros((256, 1600, 4), dtype=np.float32) # float32 is V.Imp
    # 4:class 1～4 (ch:0～3)
    ## TODO: REMOVE
    print(f'In make_mask function.\nLabels: {labels}\tFname: {fname}\nRow: {df.iloc[row_id]}')
    ##
    for idx, label in enumerate(labels.values):
        if label is not np.nan:
            label = label.split(" ")
            positions = map(int, label[0::2])
            length = map(int, label[1::2])
            mask = np.zeros(256 * 1600, dtype=np.uint8)
            for pos, le in zip(positions, length):
                mask[pos:(pos + le)] = 1
            masks[:, :, idx] = mask.reshape(256, 1600, order='F')
    return fname, masks

## Dataloader

In [0]:
class SteelDataset(Dataset):
    def __init__(self, df, data_folder, mean, std, phase):
        self.df = df
        self.phase = phase
        self.root = data_folder
        self.fnames = self.df.index.tolist()
        
        # normalize
        self.normalize = transforms.Normalize(mean=mean, std=std)

    def __getitem__(self, idx):
        image_id, mask = make_mask(idx, self.df)
        image_path = os.path.join(self.root, "train_images",  image_id)
        img = cv2.imread(image_path)
        img, mask = self.transform(phase, img=img, mask=mask)
        print(mask.shape, img.shape)
        return img, mask

    def __len__(self):
        return len(self.fnames)

    def transform(self, phase, img, mask):
        # To PIL Image
        img = Ftrans.to_pil_image(img)
        mask = Ftrans.to_pil_image(np.uint8(mask))

        # Resize
        resize = transforms.Resize(size=(256, 256))
        img = resize(img)
        mask = resize(mask)

        if phase=='train':
            # # Random Crop
            if random.random() > 0.50:
                i, j, h, w = transforms.RandomCrop.get_params(img, output_size=(256, 256))
                img = Ftrans.crop(img, i, j, h, w)
                mask = Ftrans.crop(mask ,i, j, h, w)

            # # Random Horizontal Flipping
            if random.random() > 0.50:
                img = Ftrans.hflip(img)
                mask = Ftrans.hflip(mask)

            # # Random Vertical Flipping
            if random.random() > 0.50:
                img = Ftrans.vflip(img)
                mask = Ftrans.vflip(mask)

        # # To tensor
        img = Ftrans.to_tensor(img)
        mask = Ftrans.to_tensor(mask)

        # # Normalize
        img = self.normalize(img)
        
        return img, mask

def provider(data_folder, df_path, phase, mean=None, std=None, batch_size=8, num_workers=4):
    '''Returns dataloader for the model training'''
    df = pd.read_csv(df_path)
    # https://www.kaggle.com/amanooo/defect-detection-starter-u-net
    df['ImageId'], df['ClassId'] = zip(*df['ImageId_ClassId'].str.split('_'))
    df['ClassId'] = df['ClassId'].astype(int)
    df = df.pivot(index='ImageId',columns='ClassId',values='EncodedPixels')
    df['defects'] = df.count(axis=1)
    
    train_df, val_df = train_test_split(df, test_size=test_percent, stratify=df["defects"], random_state=seed)
    df = train_df if phase == "train" else val_df
    image_dataset = SteelDataset(df, data_folder, mean, std, phase)
    dataloader = DataLoader(image_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, shuffle=True)

    return dataloader

## Metrics: IoU and Dice 

In [0]:
def metric(probability, truth, threshold=0.5, reduction='none'):
    '''Calculates dice of positive and negative images seperately'''
    '''probability and truth must be torch tensors'''
    batch_size = len(truth)
    with torch.no_grad():
        probability = probability.view(batch_size, -1)
        truth = truth.view(batch_size, -1)
        assert(probability.shape == truth.shape)

        p = (probability > threshold).float()
        t = (truth > 0.5).float()

        t_sum = t.sum(-1)
        p_sum = p.sum(-1)
        neg_index = torch.nonzero(t_sum == 0)
        pos_index = torch.nonzero(t_sum >= 1)

        dice_neg = (p_sum == 0).float()
        dice_pos = 2 * (p*t).sum(-1)/((p+t).sum(-1))

        dice_neg = dice_neg[neg_index]
        dice_pos = dice_pos[pos_index]
        dice = torch.cat([dice_pos, dice_neg])

        dice_neg = np.nan_to_num(dice_neg.mean().item(), 0)
        dice_pos = np.nan_to_num(dice_pos.mean().item(), 0)
        dice = dice.mean().item()

        num_neg = len(neg_index)
        num_pos = len(pos_index)

    return dice, dice_neg, dice_pos, num_neg, num_pos


def compute_ious(pred, label, classes, ignore_index=255, only_present=True):
    '''computes iou for one ground truth mask and predicted mask'''
    pred[label == ignore_index] = 0
    ious = []
    for c in classes:
        label_c = label == c
        if only_present and np.sum(label_c) == 0:
            ious.append(np.nan)
            continue
        pred_c = pred == c
        intersection = np.logical_and(pred_c, label_c).sum()
        union = np.logical_or(pred_c, label_c).sum()
        if union != 0:
            ious.append(intersection / union)
    return ious if ious else [1]


def compute_iou_batch(outputs, labels, classes=None):
    '''computes mean iou for a batch of ground truth masks and predicted masks'''
    ious = []
    preds = np.copy(outputs) # copy is imp
    labels = np.array(labels) # tensor to np
    for pred, label in zip(preds, labels):
        ious.append(np.nanmean(compute_ious(pred, label, classes)))
    iou = np.nanmean(ious)
    return iou

class Meter:
    '''A meter to keep track of iou and dice scores throughout an epoch'''
    def __init__(self, phase, epoch):
        self.base_threshold = 0.5 # <<<<<<<<<<< here's the threshold
        self.base_dice_scores = []
        self.dice_neg_scores = []
        self.dice_pos_scores = []
        self.iou_scores = []

    def update(self, targets, outputs):
        probs = torch.sigmoid(outputs)
        dice, dice_neg, dice_pos, _, _ = metric(probs, targets, self.base_threshold)
        self.base_dice_scores.append(dice)
        self.dice_pos_scores.append(dice_pos)
        self.dice_neg_scores.append(dice_neg)
        preds = predict(probs, self.base_threshold)
        iou = compute_iou_batch(preds, targets, classes=[1])
        self.iou_scores.append(iou)

    def get_metrics(self):
        dice = np.mean(self.base_dice_scores)
        dice_neg = np.mean(self.dice_neg_scores)
        dice_pos = np.mean(self.dice_pos_scores)
        dices = [dice, dice_neg, dice_pos]
        iou = np.nanmean(self.iou_scores)
        return dices, iou

## Logger Functions

In [0]:
def epoch_log(phase, epoch, epoch_loss, meter, start):
    '''logging the metrics at the end of an epoch'''
    dices, iou = meter.get_metrics()
    dice, dice_neg, dice_pos = dices
    print("Loss: %0.4f | IoU: %0.4f | dice: %0.4f | dice_neg: %0.4f | dice_pos: %0.4f" % (epoch_loss, iou, dice, dice_neg, dice_pos))
    return dice, iou

In [0]:
def predict(X, threshold):
    '''X is sigmoid output of the model'''
    X_p = np.copy(X)
    preds = (X_p > threshold).astype('uint8')
    return preds

## Load the data

In [0]:
phases = ["train", "val"]
dataloaders = {phase: provider(data_folder='data', df_path='data/train.csv', phase=phase,
                               mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), 
                               batch_size=batch_size, num_workers=num_workers) for phase in phases}
losses = {phase: [] for phase in phases}
iou_scores = {phase: [] for phase in phases}
dice_scores = {phase: [] for phase in phases}

## Model

In [0]:
import sys
try:
    from urllib import urlretrieve
except ImportError:
    from urllib.request import urlretrieve

def load_url(url, model_dir='./pretrained', map_location=None):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    filename = url.split('/')[-1]
    cached_file = os.path.join(model_dir, filename)
    if not os.path.exists(cached_file):
        sys.stderr.write('Downloading: "{}" to {}\n'.format(url, cached_file))
        urlretrieve(url, cached_file)
    return torch.load(cached_file, map_location=map_location)

In [0]:
class PSPModule(nn.Module):
    def __init__(self, features, out_features=1024, sizes=(1, 2, 3, 6)):
        super().__init__()
        self.stages = []
        self.stages = nn.ModuleList([self._make_stage(features, size) for size in sizes])
        self.bottleneck = nn.Conv2d(features * (len(sizes) + 1), out_features, kernel_size=1)
        self.relu = nn.ReLU()

    def _make_stage(self, features, size):
        prior = nn.AdaptiveAvgPool2d(output_size=(size, size))
        conv = nn.Conv2d(features, features, kernel_size=1, bias=False)
        return nn.Sequential(prior, conv)

    def forward(self, feats):
        h, w = feats.size(2), feats.size(3)
        priors = [F.upsample(input=stage(feats), size=(h, w), mode='bilinear') for stage in self.stages] + [feats]
        bottle = self.bottleneck(torch.cat(priors, 1))
        return self.relu(bottle)


class PSPUpsample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.PReLU()
        )

    def forward(self, x):
        h, w = 2 * x.size(2), 2 * x.size(3)
        p = F.upsample(input=x, size=(h, w), mode='bilinear')
        return self.conv(p)


class PSPNet(nn.Module):
    def __init__(self, n_classes=18, sizes=(1, 2, 3, 6), psp_size=2048, deep_features_size=1024, backend='resnet34',
                 pretrained=True):
        super().__init__()
        self.feats = getattr(extractors, backend)(pretrained)
        self.psp = PSPModule(psp_size, 1024, sizes)
        self.drop_1 = nn.Dropout2d(p=0.3)

        self.up_1 = PSPUpsample(1024, 256)
        self.up_2 = PSPUpsample(256, 64)
        self.up_3 = PSPUpsample(64, 64)

        self.drop_2 = nn.Dropout2d(p=0.15)
        self.final = nn.Sequential(
            nn.Conv2d(64, n_classes, kernel_size=1),
            nn.LogSoftmax()
        )

        self.classifier = nn.Sequential(
            nn.Linear(deep_features_size, 256),
            nn.ReLU(),
            nn.Linear(256, n_classes)
        )

    def forward(self, x):
        f, class_f = self.feats(x) 
        p = self.psp(f)
        p = self.drop_1(p)

        p = self.up_1(p)
        p = self.drop_2(p)

        p = self.up_2(p)
        p = self.drop_2(p)

        p = self.up_3(p)
        p = self.drop_2(p)

        auxiliary = F.adaptive_max_pool2d(input=class_f, output_size=(1, 1)).view(-1, class_f.size(1))

        return self.final(p), self.classifier(auxiliary)

In [0]:
milestones = [15, 30, 45]

net = PSPNet(n_classes=4, sizes=(1, 2, 3, 6), psp_size=2048, deep_features_size=1024, backend='resnet50')
net = net.to(device)

criterion = nn.NLLLoss(ignore_index=-1)
# cls_criterion = nn.BCEWithLogitsLoss(weight=class_weights)
optimizer = optim.Adam(net.parameters(), lr=1e-2, weight_decay=1e-4)
scheduler = MultiStepLR(optimizer, milestones=[int(x) for x in milestones.split(',')])

## Training and Validation

In [0]:
best_loss = float('inf')
print('Training Started...')
for epoch in range(num_epochs):
    for phase in phases:
        running_loss = 0.0
        meter = Meter(phase, epoch)
        start = time.strftime("%H:%M:%S")
        
        print(f"Starting epoch: {epoch} | phase: {phase} | ⏰: {start}")
        
        # Each epoch has a training and validation phase
        if phase=="train":
            net.train()
        else:
            net.eval()
        
        #
        net.zero_grad()
        for itr, batch in enumerate(dataloaders[phase]):
            images, masks = batch
            images = images.to(device)
            masks = masks.to(device)
            
            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase=='train'):
                outputs = net_decoder(net_encoder(images))
                print(f'encoder:{net_encoder(images).shape}, decoder:{net_decoder(net_encoder(images)).shape}')
                loss = criterion(masks, outputs)
                loss /= accumulation_steps

                if phase=='train':
                    loss.backward()
                    if (itr+1)%accumulation_steps == 0:
                        encoder_optimizer.step()
                        net.zero_grad()
            
            running_loss += loss.item()
            outputs = outputs.detach().cpu()
            meter.update(masks.cpu(), outputs)
    
        epoch_loss = (running_loss * accumulation_steps) / total_batches
        dice, iou = epoch_log(phase, epoch, epoch_loss, meter, start)
        losses[phase].append(epoch_loss)
        dice_scores[phase].append(dice)
        iou_scores[phase].append(iou)
        torch.cuda.empty_cache()
        
        state = {
            "epoch": epoch,
            "best_loss": best_loss,
            "encoder_state_dict": net_encoder.state_dict(),
            "decoder_state_dict": net_decoder.state_dict(),
            "encoder_optimizer": encoder_optimizer.state_dict(),
            "decoder_optimizer": decoder_optimizer.state_dict()
        }
        
        if phase=='valid':
            encoder_scheduler.step(epoch_loss)
            decoder_scheduler.step(epoch_loss)
            if epoch_loss < best_loss:
                print("******** New optimal found, saving state ********")
                state["best_loss"] = best_loss = val_epoch
                torch.save(state, f"./model_{epoch}_{start}.pth")
    print() 

Training Started...
Starting epoch: 0 | phase: train | ⏰: 09:51:06
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256])
torch.Size([4, 256, 256]) torch.Size([3, 256, 256

TypeError: ignored

In [0]:
# class Trainer(object):
#     '''This class takes care of training and validation of our model'''
#     def __init__(self, model):
#         self.best_loss = float("inf")
#         self.phases = ["train", "val"]
#         self.net = model 
#         self.net = self.net.to(self.device)
#         cudnn.benchmark = True
#         self.dataloaders = {
#             phase: provider(
#                 data_folder=data_folder,
#                 df_path=train_df_path,
#                 phase=phase,
#                 mean=(0.485, 0.456, 0.406),
#                 std=(0.229, 0.224, 0.225),
#                 batch_size=self.batch_size[phase],
#                 num_workers=self.num_workers,
#             )
#             for phase in self.phases
#         }
#         self.losses = {phase: [] for phase in self.phases}
#         self.iou_scores = {phase: [] for phase in self.phases}
#         self.dice_scores = {phase: [] for phase in self.phases}
        
#     def forward(self, images, targets):
#         images = images.to(self.device)
#         masks = targets.to(self.device)
#         outputs = self.net(images)
#         loss = self.criterion(outputs, masks)
#         return loss, outputs

#     def iterate(self, epoch, phase):
#         meter = Meter(phase, epoch)
#         start = time.strftime("%H:%M:%S")
#         print(f"Starting epoch: {epoch} | phase: {phase} | ⏰: {start}")
#         batch_size = self.batch_size[phase]
#         self.net.train(phase == "train")
#         dataloader = self.dataloaders[phase]
#         running_loss = 0.0
#         total_batches = len(dataloader)
# #         tk0 = tqdm(dataloader, total=total_batches)
#         self.optimizer.zero_grad()
#         for itr, batch in enumerate(dataloader): # replace `dataloader` with `tk0` for tqdm
#             images, targets = batch
#             loss, outputs = self.forward(images, targets)
#             loss = loss / self.accumulation_steps
#             if phase == "train":
#                 loss.backward()
#                 if (itr + 1 ) % self.accumulation_steps == 0:
#                     self.optimizer.step()
#                     self.optimizer.zero_grad()
#             running_loss += loss.item()
#             outputs = outputs.detach().cpu()
#             meter.update(targets, outputs)
# #             tk0.set_postfix(loss=(running_loss / ((itr + 1))))
#         epoch_loss = (running_loss * self.accumulation_steps) / total_batches
#         dice, iou = epoch_log(phase, epoch, epoch_loss, meter, start)
#         self.losses[phase].append(epoch_loss)
#         self.dice_scores[phase].append(dice)
#         self.iou_scores[phase].append(iou)
#         torch.cuda.empty_cache()
#         return epoch_loss

#     def start(self):
#         for epoch in range(self.num_epochs):
#             self.iterate(epoch, "train")
#             state = {
#                 "epoch": epoch,
#                 "best_loss": self.best_loss,
#                 "state_dict": self.net.state_dict(),
#                 "optimizer": self.optimizer.state_dict(),
#             }
#             with torch.no_grad():
#                 val_loss = self.iterate(epoch, "val")
#                 self.scheduler.step(val_loss)
#             if val_loss < self.best_loss:
#                 print("******** New optimal found, saving state ********")
#                 state["best_loss"] = self.best_loss = val_loss
#                 torch.save(state, "./model.pth")
#             print()

In [0]:
# model_trainer = Trainer(model)
# model_trainer.start()

## Plotting

In [0]:
# PLOT TRAINING
# losses = model_trainer.losses
# dice_scores = model_trainer.dice_scores # overall dice
# iou_scores = model_trainer.iou_scores

def plot(scores, name):
    plt.figure(figsize=(15,5))
    plt.plot(range(len(scores["train"])), scores["train"], label=f'train {name}')
    plt.plot(range(len(scores["train"])), scores["val"], label=f'val {name}')
    plt.title(f'{name} plot'); plt.xlabel('Epoch'); plt.ylabel(f'{name}');
    plt.legend(); 
    plt.show()

plot(losses, "BCE loss")
plot(dice_scores, "Dice score")
plot(iou_scores, "IoU score")

NameError: ignored